### Herald Scraper

This notebook ncludes the code needed to scrape articles from [The Herald](https://www.nzherald.co.nz/)

In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import time
from tqdm import tqdm

In [2]:
url = 'https://www.nzherald.co.nz/business/news/headlines.cfm?c_id=3'

In [3]:
r1 = requests.get(url)
print(r1.status_code)

coverpage = r1.content

soup1 = BeautifulSoup(coverpage, 'html5lib')

# coverpage_news = soup1.find_all('a', href=lambda href: href and 'business' in href)
# coverpage_news = soup1.find_all('article', class_=lambda x: x != 'marketing---text-below')
coverpage_news = soup1.find_all('article')
len(coverpage_news)

200


120

In [127]:
news_contents = []
news_dates = []
list_links = []
list_titles = []

for n in tqdm(np.arange(0, len(coverpage_news))):
        
    # Getting the link of the article
    link = coverpage_news[n].find('a')['href']
    list_links.append(link)
    
    # Reading the content (it is divided in paragraphs)
    try:
        article = requests.get('https://www.nzherald.co.nz'+str(link))
    except:
        article = 0
    if article != 0:
        article_content = article.content
        soup_article = BeautifulSoup(article_content, 'html5lib')
        article_date = soup_article.find_all('div', class_='publish')
        body = soup_article.find_all('div', id='article-body')
        if body != []:
            x = body[0].find_all('p')
            for child in article_date[0].find_all("div"):
                child.decompose()
            article_date = article_date[0].get_text()
            # Unifying the paragraphs
            list_paragraphs = []
            try:
                for p in np.arange(0, len(x)):
                    paragraph = x[p].get_text()
                    list_paragraphs.append(paragraph)
                    final_article = " ".join(list_paragraphs)
            except:
#                 print('no info')
                pass

    else:
#         print('not an article')
        final_article = np.nan
        article_date = np.nan
    news_contents.append(final_article)
    news_dates.append(article_date)

100%|██████████| 120/120 [02:50<00:00,  1.42s/it]


In [149]:
df_features = pd.DataFrame({'Article Content': news_contents, 'Article Link': list_links, 'Article Date': news_dates})

In [150]:
df_features = df_features.dropna(axis=0)
df_features = df_features.drop_duplicates(subset ='Article Content', keep='first')
df_features = df_features.reset_index(drop=True)

In [159]:
now=pd.datetime.now()
df_features.to_pickle('herald_business_{}_{}-{}.pkl'.format(now.day, now.month, now.year))

In [158]:
pd.datetime.now()

datetime.datetime(2019, 11, 23, 19, 12, 51, 286754)